# Capstone Project - The Battle of Neighborhoods (Week 1-2)

## Introduction/Business Problem

### Introduction


Colombia is one of the largest meat consumers in Latin America with an estimated consumption of 18.6 kilograms in 2019. Despite this, compared to Latin America it is in the top 6 of the largest consumers in the region. Likewise, Colombia is a livestock country, despite the fact its exports remain well below its competitors. Colombian culture, tradition and fields are conducive to raising cattle and at the same time being an attractive market within the country.

### Business Problem
In this case there is a contractor, stakeholder intered to open Butcher shop in Colombia, exactly in Bogotá or surroundings i.e Cundinamarca but he wants to be as close as possible to their customers since they want to offer their products to restaurants and steakhouses in order to focus on this niche. However, the stakeholder does not know Bogota and its surroundings, that is why through this project it will be revealed where you can establish his business.

To solve this problem we will cluster the cities and neighborhoods of Bogota that show the venues that have the most visits to places such as restaurants and steakhouses and from this we will recommend the best place to establish the butcher shop.

### Data
For this project we wil use the following data :

First we must obtain the data of the coordinates of the neighborhoods of Bogotá and the cities of Cundinamarca since this is the target area of the interested party, for them we will use the following page https://www.geonames.org/


In order to explore and find venues in each city and neighborhood we will use the FourSquare API as well as classify the most common venues for each one ans then we will be able to recommend the best place to be stablished.

### Methodology 

- Collect Bogotá neighborhoods and Cundinamarca cities Data thanks to https://www.geonames.org/
- Explore and size data
- Filter veneus and get top 5 most common venues
- Modeling and visualization

In [1]:
!pip install geopy
from geopy.geocoders import Nominatim
import geopy
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
! pip install folium==0.5.0
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 79 kB 5.6 MB/s eta 0:00:011
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=f267d75c392f84bf73a11f5be908171aa1c56f63a01d55defe87b3a6c262e1ab
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Libraries imported.


### IMPORTING DATA

In [5]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_dadb02c374eb45a78a6b179466548cf4 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xkAdChVTcQCt0Syi0K5BnArlJbnvJiX2qS1uZZFFqUV_',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_dadb02c374eb45a78a6b179466548cf4.get_object(Bucket='courseracapstone-donotdelete-pr-bz7unjxzn6dub3',Key='data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body,sep=';')
df.head()


,City,State,Latitude,Longitude
0,Agua De Dios,Cundinamarca,4.3765,-74.46700
1,Agua De Dios,Cundinamarca,4.3765,-74.46700
2,Alban,Cundinamarca,4.8766,-74.44377
3,Alban,Cundinamarca,4.8766,-74.44377
4,Anapoima,Cundinamarca,4.5203,-74.45394


In [6]:
df.shape

(376, 4)

In [7]:
## Define Foursquare Credentials and Version
CLIENT_ID = 'CONIN1VSHX2IAUQ31ZO0MFG5ONE044UXLI42ZUZKT2GS5MSX' # Foursquare ID
CLIENT_SECRET = 'T0JFYKC4QFBYGSL5IUNYFUTJWS301GXC3OPJ5HYFKOXJBTM0' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CONIN1VSHX2IAUQ31ZO0MFG5ONE044UXLI42ZUZKT2GS5MSX
CLIENT_SECRET:T0JFYKC4QFBYGSL5IUNYFUTJWS301GXC3OPJ5HYFKOXJBTM0


In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

###  FINDING THE VENUES OF BOGOTÁ AND CUNDINAMARCA

In [9]:
Cun_Bog_venues = getNearbyVenues(names=df['City'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Agua De Dios
Agua De Dios
Alban
Alban
Anapoima
Anapoima
Anapoima
Anolaima
Anolaima
Anolaima
Arbelaez
Arbelaez
Arbelaez
Beltran
Beltran
Bituima
Bituima
Bojaca
Bojaca
Cabrera
Cabrera
Cachipay
Cachipay
Cajica
Cajica
Caparrapi
Caparrapi
Caparrapi
Caqueza
Caqueza
Carmen De Carupa
Carmen De Carupa
Carmen De Carupa
Chaguani
Chaguani
Chia
Chia
Chia
Chia
Chipaque
Chipaque
Chipaque
Choachi
Choachi
Choachi
Choconta
Choconta
Choconta
Cogua
Cogua
Cogua
Cota
Cota
Cucunuba
Cucunuba
El Colegio
El Colegio
El Colegio
El Peñon
El Peñon
El Rosal
El Rosal
Facatativa
Facatativa
Facatativa
Facatativa
Fomeque
Fomeque
Fomeque
Fosca
Fosca
Funza
Funza
Fuquene
Fuquene
Fusagasuga
Fusagasuga
Fusagasuga
Fusagasuga
Fusagasuga
Gachala
Gachala
Gachancipa
Gachancipa
Gacheta
Gacheta
Gacheta
Gama
Gama
Girardot
Girardot
Girardot
Granada
Granada
Guacheta
Guacheta
Guacheta
Guaduas
Guaduas
Guaduas
Guaduas
Guasca
Guasca
Guasca
Guataqui
Guataqui
Guatavita
Guatavita
Guayabal De Siquima
Guayabal De Siquima
Guayabetal
Guayabetal
G

In [11]:
Cun_Bog_venues.shape

(60, 7)

In [10]:
Cun_Bog_venues.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bojaca,4.7318,-74.43413,Cachipay,4.730386,-74.436754,General Travel
1,Bojaca,4.7318,-74.43413,La Parrillada de Alejo,4.730920,-74.435590,Fast Food Restaurant
2,Bojaca,4.7318,-74.43413,Parque Principal Cachipay,4.730805,-74.436977,Plaza
3,Bojaca,4.7318,-74.43413,Cachipay,4.730386,-74.436754,General Travel
4,Bojaca,4.7318,-74.43413,La Parrillada de Alejo,4.730920,-74.435590,Fast Food Restaurant


In [12]:
print('There are {} uniques categories.'.format(len(Cun_Bog_venues['Venue Category'].unique())))

There are 21 uniques categories.


In [16]:
Cun_Bog_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Barrios Unidos,4,4,4,4,4,4
Bojaca,6,6,6,6,6,6
Chapinero,5,5,5,5,5,5
Choconta,3,3,3,3,3,3
Fontibon,1,1,1,1,1,1
Guacheta,6,6,6,6,6,6
Kennedy,7,7,7,7,7,7
Macheta,6,6,6,6,6,6
Madrid,3,3,3,3,3,3


In [20]:
# one hot encoding
Cun_Bog_venues_onehot = pd.get_dummies(Cun_Bog_venues[['Venue Category']], prefix="", prefix_sep="")
#Cun_Bog_venues_onehot.drop(['City'],axis=1,inplace=True) 
Cun_Bog_venues_onehot.insert(loc=0, column='City', value=Cun_Bog_venues['City'] )
Cun_Bog_venues_onehot.shape

(60, 22)

In [21]:
Cun_Bog_grouped = Cun_Bog_venues_onehot.groupby('City').mean().reset_index()
Cun_Bog_grouped.head()

,City,BBQ Joint,Bakery,Campground,Cupcake Shop,Farm,Fast Food Restaurant,Food,Gastropub,General Travel,Hotel,Italian Restaurant,Mexican Restaurant,Pizza Place,Plaza,Resort,Restaurant,Scenic Lookout,Sculpture Garden,Shopping Mall,South American Restaurant,Steakhouse
0,Barrios Unidos,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.25,0.000000,0.0,0.0,0.25,0.0,0.000000,0.0,0.25,0.0,0.0,0.0,0.0,0.25
1,Bojaca,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.00,0.333333,0.0,0.0,0.00,0.0,0.333333,0.0,0.00,0.0,0.0,0.0,0.0,0.00
2,Chapinero,0.2,0.0,0.0,0.0,0.2,0.000000,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.60,0.0,0.0,0.0,0.0,0.00
3,Choconta,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,0.000000,1.0,0.00,0.0,0.0,0.0,0.0,0.00
4,Fontibon,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,1.00,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.00


In [22]:
Cun_Bog_grouped.shape

(17, 22)

In [24]:
num_top_venues = 5

for hood in Cun_Bog_grouped['City']:
    print("----"+hood+"----")
    temp = Cun_Bog_grouped[Cun_Bog_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barrios Unidos----
                       venue  freq
0                 Steakhouse  0.25
1         Mexican Restaurant  0.25
2                 Restaurant  0.25
3                  Gastropub  0.25
4  South American Restaurant  0.00


----Bojaca----
                  venue  freq
0  Fast Food Restaurant  0.33
1        General Travel  0.33
2                 Plaza  0.33
3             BBQ Joint  0.00
4           Pizza Place  0.00


----Chapinero----
                       venue  freq
0                 Restaurant   0.6
1                  BBQ Joint   0.2
2                       Farm   0.2
3         Mexican Restaurant   0.0
4  South American Restaurant   0.0


----Choconta----
                       venue  freq
0                     Resort   1.0
1                  BBQ Joint   0.0
2         Mexican Restaurant   0.0
3  South American Restaurant   0.0
4              Shopping Mall   0.0


----Fontibon----
                       venue  freq
0         Mexican Restaurant   1.0
1                  BBQ

In [113]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### difining the top 5 common venues for each city or neighborhood

In [136]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
City_venues_sorted = pd.DataFrame(columns=columns)
City_venues_sorted['City'] = Cun_Bog_grouped['City']

for ind in np.arange(Cun_Bog_grouped.shape[0]):
    City_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Cun_Bog_grouped.iloc[ind, :], num_top_venues)

City_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barrios Unidos,Steakhouse,Restaurant,Mexican Restaurant,Gastropub,General Travel
1,Bojaca,Fast Food Restaurant,Plaza,General Travel,Steakhouse,Hotel
2,Chapinero,Restaurant,BBQ Joint,Farm,Hotel,Bakery
3,Choconta,Resort,Steakhouse,Hotel,Bakery,Campground
4,Fontibon,Mexican Restaurant,Steakhouse,Hotel,Bakery,Campground


### SETTING CLUSTERS

In [137]:
# set number of clusters
kclusters = 5

Cun_Bog_grouped_clustering = Cun_Bog_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Cun_Bog_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 3, 2, 2, 2, 2, 2, 0], dtype=int32)

In [138]:
# add clustering labels
City_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Cun_Bog_merged = df

Cun_Bog_merged = Cun_Bog_merged.join(City_venues_sorted.set_index('City'), on='City')

Cun_Bog_merged.head() # check the last columns!

,City,State,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agua De Dios,Cundinamarca,4.3765,-74.46700,NaN,NaN,NaN,NaN,NaN,NaN
1,Agua De Dios,Cundinamarca,4.3765,-74.46700,NaN,NaN,NaN,NaN,NaN,NaN
2,Alban,Cundinamarca,4.8766,-74.44377,NaN,NaN,NaN,NaN,NaN,NaN
3,Alban,Cundinamarca,4.8766,-74.44377,NaN,NaN,NaN,NaN,NaN,NaN
4,Anapoima,Cundinamarca,4.5203,-74.45394,NaN,NaN,NaN,NaN,NaN,NaN


### ELIMINATING NAN DATA

In [144]:
Cun_Bog_merged=Cun_Bog_merged.dropna()
Cun_Bog_merged

,City,State,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,Bojaca,Cundinamarca,4.7318,-74.43413,2.0,Fast Food Restaurant,Plaza,General Travel,Steakhouse,Hotel
18,Bojaca,Cundinamarca,4.7318,-74.43413,2.0,Fast Food Restaurant,Plaza,General Travel,Steakhouse,Hotel
45,Choconta,Cundinamarca,5.1447,-73.36858,3.0,Resort,Steakhouse,Hotel,Bakery,Campground
46,Choconta,Cundinamarca,5.1447,-73.36858,3.0,Resort,Steakhouse,Hotel,Bakery,Campground
47,Choconta,Cundinamarca,5.1447,-73.36858,3.0,Resort,Steakhouse,Hotel,Bakery,Campground
94,Guacheta,Cundinamarca,5.3842,-73.36862,2.0,BBQ Joint,Pizza Place,Hotel,Bakery,Campground
95,Guacheta,Cundinamarca,5.3842,-73.36862,2.0,BBQ Joint,Pizza Place,Hotel,Bakery,Campground
96,Guacheta,Cundinamarca,5.3842,-73.36862,2.0,BBQ Joint,Pizza Place,Hotel,Bakery,Campground
136,Macheta,Cundinamarca,5.0815,-73.36076,2.0,Bakery,Sculpture Garden,Steakhouse,Hotel,Campground
137,Macheta,Cundinamarca,5.0815,-73.36076,2.0,Bakery,Sculpture Garden,Steakhouse,Hotel,Campground


### CLUSTERS MAP

In [143]:
# create map
map_clusters = folium.Map(location=[4.598889, -74.0808], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, city, cluster in zip(Cun_Bog_merged['Latitude'], Cun_Bog_merged['Longitude'], Cun_Bog_merged['City'], Cun_Bog_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [145]:
## Examine Clusters

## Cluster 1
Cun_Bog_merged.loc[Cun_Bog_merged['Cluster Labels'] == 0, Cun_Bog_merged.columns[[0] + list(range(5, Cun_Bog_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
147,Mosquera,Farm,Steakhouse,Hotel,Bakery,Campground
148,Mosquera,Farm,Steakhouse,Hotel,Bakery,Campground
196,San Francisco,Farm,Steakhouse,Hotel,Bakery,Campground
197,San Francisco,Farm,Steakhouse,Hotel,Bakery,Campground
198,San Francisco,Farm,Steakhouse,Hotel,Bakery,Campground
216,Soacha,Farm,Steakhouse,Hotel,Bakery,Campground
217,Soacha,Farm,Steakhouse,Hotel,Bakery,Campground
218,Soacha,Farm,Steakhouse,Hotel,Bakery,Campground
219,Soacha,Farm,Steakhouse,Hotel,Bakery,Campground
220,Soacha,Farm,Steakhouse,Hotel,Bakery,Campground


In [146]:
## Cluster 2
Cun_Bog_merged.loc[Cun_Bog_merged['Cluster Labels'] == 1, Cun_Bog_merged.columns[[0] + list(range(5, Cun_Bog_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
352,Teusaquillo,Campground,Steakhouse,Hotel,Bakery,Cupcake Shop
353,Teusaquillo,Campground,Steakhouse,Hotel,Bakery,Cupcake Shop


In [149]:
## Cluster 3
Cun_Bog_merged.loc[Cun_Bog_merged['Cluster Labels'] == 2, Cun_Bog_merged.columns[[0] + list(range(5, Cun_Bog_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,Bojaca,Fast Food Restaurant,Plaza,General Travel,Steakhouse,Hotel
18,Bojaca,Fast Food Restaurant,Plaza,General Travel,Steakhouse,Hotel
94,Guacheta,BBQ Joint,Pizza Place,Hotel,Bakery,Campground
95,Guacheta,BBQ Joint,Pizza Place,Hotel,Bakery,Campground
96,Guacheta,BBQ Joint,Pizza Place,Hotel,Bakery,Campground
136,Macheta,Bakery,Sculpture Garden,Steakhouse,Hotel,Campground
137,Macheta,Bakery,Sculpture Garden,Steakhouse,Hotel,Campground
138,Macheta,Bakery,Sculpture Garden,Steakhouse,Hotel,Campground
139,Madrid,Italian Restaurant,Hotel,Bakery,Campground,Cupcake Shop
140,Madrid,Italian Restaurant,Hotel,Bakery,Campground,Cupcake Shop


In [150]:
## Cluster 4
Cun_Bog_merged.loc[Cun_Bog_merged['Cluster Labels'] == 3, Cun_Bog_merged.columns[[0] + list(range(5, Cun_Bog_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
45,Choconta,Resort,Steakhouse,Hotel,Bakery,Campground
46,Choconta,Resort,Steakhouse,Hotel,Bakery,Campground
47,Choconta,Resort,Steakhouse,Hotel,Bakery,Campground


In [151]:
## Cluster 5
Cun_Bog_merged.loc[Cun_Bog_merged['Cluster Labels'] == 4, Cun_Bog_merged.columns[[0] + list(range(5, Cun_Bog_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
295,Usaquen,Hotel,Steakhouse,Bakery,Campground,Cupcake Shop
296,Usaquen,Hotel,Steakhouse,Bakery,Campground,Cupcake Shop
297,Usaquen,Hotel,Steakhouse,Bakery,Campground,Cupcake Shop
298,Usaquen,Hotel,Steakhouse,Bakery,Campground,Cupcake Shop
299,Usaquen,Hotel,Steakhouse,Bakery,Campground,Cupcake Shop
340,Suba,Hotel,Steakhouse,Bakery,Campground,Cupcake Shop
341,Suba,Hotel,Steakhouse,Bakery,Campground,Cupcake Shop
342,Suba,Hotel,Steakhouse,Bakery,Campground,Cupcake Shop
343,Suba,Hotel,Steakhouse,Bakery,Campground,Cupcake Shop
344,Suba,Hotel,Steakhouse,Bakery,Campground,Cupcake Shop


### Results section and Discussion section 
Examining the final results that were carried out, it can be observed that the most frequented places in the cities of Cundinamarca and Bogotá are gastronomic places, being the main objective of this project.
The results show the cluster 5 as the most interesting cluster since the gastronomic sector is only below the hotel sector as the most common in the area, as well as having more representations. However,the 3 cluster shows us Barrios Unidos with the steakhouse category as the most common, although they are not so representative within the same cluster

### Conclusion section
We explore a huge area in Colombia taking into account the capial of Colomia, Bogotá and one of the largest deparments in the country withe the purpose of find the best place to stablish a butcher shop 

To solve this business problem we clustered cities of cundinamarca and neighborhoods of Bogotá, find the most commonn venues and cluster them and giving as the most indicated place that one which form the biggest cluster and at least restaurants pr steakhouses be the most common venues.

we drew the conclusion that Usaquen and Suba are the best places to stablish a butcher shop. they are located at the north of Bogotá and form as per previous results the most visited gastronomic areas of Bogota and so the stakeholder can be near of their future clients 